## Update  1/14/2023
1. Execute the code cell below.
2. After running the cell, restart your session.
- **Reason:** Currently, a session restart is required for the model download. This might be a bug and could be patched in future releases.


In [ ]:
!pip install peft
!pip install transformers
!pip install bitsandbytes
!pip install accelerate

### You need a huggingface account that has access to llama2

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
device = "cuda" if torch.cuda.is_available() else "cpu"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

peft_model_id = "danjie/Chadgpt-Llama2-70b-conversation"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map='auto', quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
## Run this cell to start a new conversation
conversation_history = []

def format_conversation(conversation: list[str]) -> str:
    formatted_conversation = ""

    # Check if the conversation has more than two turns
    if len(conversation) > 2:
        # Process all but the last two turns
        for i in range(len(conversation) - 2):
            if i % 2 == 0:
                formatted_conversation += "<Past User>" + conversation[i] + "\n"
            else:
                formatted_conversation += "<Past Assistant>" + conversation[i] + "\n"

    # Process the last two turns
    if len(conversation) >= 2:
        formatted_conversation += "<User>" + conversation[-2] + "\n"
        formatted_conversation += "<Assistant>" + conversation[-1]

    return formatted_conversation

def talk_with_llm(chat: str) -> str:
    # Encode and move tensor into cuda if applicable.
    conversation_history.append(chat)
    conversation_history.append("")
    conversation = format_conversation(conversation_history)

    encoded_input = tokenizer(conversation, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    output = model.generate(**encoded_input, max_new_tokens=256)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response = response[len(conversation):]

    conversation_history.pop()
    conversation_history.append(response)
    return response

### Talk to llm

In [12]:
## Run this cell to start a new conversation
conversation_history = []

In [13]:
talk_with_llm("Hey buddy, could you please pick up some milk on your way home this evening?")

' No problem . '